### Test Out FuzzyPandas Functionality

In [1]:
import pandas as pd

#### Read In U.S. News Data

In [2]:
def read_us_news(pickle):
    df = pd.read_pickle(pickle)   
    return pd.concat([us_news, df], axis=0, ignore_index=True)

us_news = pd.DataFrame() #initialize empty data frame
us_news = read_us_news("example_data/us_news/usnews-ranking-national-universities.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-national-liberal-arts-colleges.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-colleges-midwest.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-colleges-north.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-colleges-south.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-colleges-west.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-universities-midwest.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-universities-north.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-universities-south.pickle")
us_news = read_us_news("example_data/us_news/usnews-ranking-regional-universities-west.pickle")

us_news["city"] = us_news.location.apply(lambda x: x.split(", ")[0].upper().strip()) #parse out city, to upper case
us_news["state"] = us_news.location.apply(lambda x: x.split(", ")[1].upper().strip()) #parse out state, to upper case
us_news["school"] = us_news.school.apply(lambda x: x.upper().strip()) #school name to upper case

us_news.drop("location", axis=1, inplace=True) #drop original location (now that we've split out into city and state)
us_news.head(5)

,category,school,score,city,state
0,National Universities,PRINCETON UNIVERSITY,100 out of 100.,PRINCETON,NJ
1,National Universities,HARVARD UNIVERSITY,99 out of 100.,CAMBRIDGE,MA
2,National Universities,YALE UNIVERSITY,97 out of 100.,NEW HAVEN,CT
3,National Universities,COLUMBIA UNIVERSITY,95 out of 100.,NEW YORK,NY
4,National Universities,STANFORD UNIVERSITY,95 out of 100.,STANFORD,CA


#### Read in NY & CT IPEDS Data (IPEDS can be downloaded for free [here](https://nces.ed.gov/ipeds/datacenter/Default.aspx))

In [3]:
ipeds=pd.read_csv("example_data/ipeds/HD2014.csv")[["UNITID","INSTNM","CITY","STABBR"]]
ipeds.rename(columns={"UNITID":"unitid","INSTNM":"school","CITY":"city","STABBR":"state"}, inplace=True) 

##Convert Keys to uppercase, convert to unicode (deal with encoding issues later)
ipeds["school"] = ipeds.school.apply(lambda x: x.upper().strip().decode("ascii", "replace"))
ipeds["city"] = ipeds.city.apply(lambda x: x.upper().strip().decode("ascii", "replace")) 
ipeds["state"] = ipeds.state.apply(lambda x: x.upper().strip().decode("ascii", "replace"))

print ipeds.shape
ipeds.head(5)

(7687, 4)


,unitid,school,city,state
0,100636,COMMUNITY COLLEGE OF THE AIR FORCE,MONTGOMERY,AL
1,100654,ALABAMA A & M UNIVERSITY,NORMAL,AL
2,100663,UNIVERSITY OF ALABAMA AT BIRMINGHAM,BIRMINGHAM,AL
3,100690,AMRIDGE UNIVERSITY,MONTGOMERY,AL
4,100706,UNIVERSITY OF ALABAMA IN HUNTSVILLE,HUNTSVILLE,AL


In [4]:
print type(us_news.school.tolist()[0])
print type(ipeds.school.tolist()[0])

<type 'unicode'>
<type 'unicode'>


#### Subset to only NY & CT schools

In [6]:
us_news=us_news[us_news.state.isin(["NY", "CT"])].copy()
ipeds=ipeds[ipeds.state.isin(["NY", "CT"])].copy()

print us_news.shape
print ipeds.shape

(142, 5)
(574, 4)


In [10]:
import fuzzypandas as fp

xwalk = brute_force_merge(us_news, ipeds, on=["state","city","school"])

ImportError: No module named fuzzypandas